<a href="https://colab.research.google.com/github/ricardoruedas/ML/blob/main/%5B05%5D%20-%20Arboles%20de%20decision/LightGMB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Arboles de decisión: LightGMB - Ejercicio 4: LightGMB.ipynb

Este notebook es un **I do**: todo resuelto y explicado paso a paso.

## Objetivos

- Creo un dataset alearotio con 50000 ejemplos para predecir si un cliente comprará.
- Entrena un LightGMB.
- Evalúa el modelo con métricas de clasificación (accuracy, matriz de confusión y reporte).
- Muestra la importancia de cada característica (qué variables usa más el modelo para decidir).
- Hacer una competición entre LightGMB VS XGBoost VS Gradient Boosting

## 1) Instalamos y cargamos librerias xgboost

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.datasets import make_classification, make_regression
from sklearn.metrics import accuracy_score, mean_squared_error, log_loss
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

# ¡IMPORTANTE! Instalar las librerías:
# pip install lightgbm xgboost
import lightgbm as lgb
import xgboost as xgb

## 2) Preparamos datos

In [2]:
# Preparamos datos

print("PREPARANDO EL CIRCUITO:")
print("-" * 50)

# Dataset grande para ver diferencias reales
np.random.seed(42)
X_large, y_large = make_classification(
    n_samples=50000,  # Dataset GRANDE para ver velocidad
    n_features=100,
    n_informative=80,
    n_redundant=10,
    n_classes=2,
    random_state=42
)

# Añadir características categóricas (ventaja para LightGBM)
categorical_features = []
for i in range(5):  # 5 características categóricas
    cat_feature = np.random.choice(['A', 'B', 'C', 'D'], size=X_large.shape[0])
    # Convertir a números para sklearn
    cat_encoded = pd.Categorical(cat_feature).codes
    X_large = np.column_stack([X_large, cat_encoded])
    categorical_features.append(X_large.shape[1] - 1)

print(f"🏁 Circuito preparado:")
print(f"   📊 Muestras: {X_large.shape[0]:,}")
print(f"   📋 Características: {X_large.shape[1]} (5 categóricas)")
print(f"   🏷️  Características categóricas: {categorical_features}")

PREPARANDO EL CIRCUITO:
--------------------------------------------------
🏁 Circuito preparado:
   📊 Muestras: 50,000
   📋 Características: 105 (5 categóricas)
   🏷️  Características categóricas: [100, 101, 102, 103, 104]


In [3]:
# Crear algunos valores faltantes
n_missing = int(0.05 * X_large.shape[0] * X_large.shape[1])  # 5% faltantes
missing_rows = np.random.choice(X_large.shape[0], size=n_missing, replace=True)
missing_cols = np.random.choice(X_large.shape[1]-5, size=n_missing, replace=True)  # No en categóricas
X_large[missing_rows, missing_cols] = np.nan

print(f"   🕳️  Valores faltantes: {np.isnan(X_large).sum():,}")

   🕳️  Valores faltantes: 255,703


In [4]:
# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(
    X_large, y_large, test_size=0.2, random_state=42
)
print(f"   🏋️  Entrenamiento: {X_train.shape[0]:,} muestras")
print(f"   🧪 Prueba: {X_test.shape[0]:,} muestras")

   🏋️  Entrenamiento: 40,000 muestras
   🧪 Prueba: 10,000 muestras


## 3) Preparamos los modelos

In [5]:
#Preparar los modelos

print("PREPARACIÓN DE COMPETIDORES:")
print("-" * 50)

# Configuración común
common_params = {
    'n_estimators': 100,
    'max_depth': 6,
    'learning_rate': 0.1,
    'random_state': 42
}

print("⚙️  Configuración común:")
print(f"   🌳 Estimadores: {common_params['n_estimators']}")
print(f"   📏 Profundidad: {common_params['max_depth']}")
print(f"   📚 Learning rate: {common_params['learning_rate']}")

# Preparar datos para Gradient Boosting (necesita imputación)
imputer = SimpleImputer(strategy='mean')
X_train_filled = imputer.fit_transform(X_train)
X_test_filled = imputer.transform(X_test)

print("\n🏎️ COMPETIDORES:")

# 1. Gradient Boosting Clásico
print("\n🥉 Competidor 1: Gradient Boosting Clásico")
print("   💪 Fortalezas: Estable, confiable, fácil")
print("   😰 Debilidades: Lento, necesita preprocesamiento")

gbm_model = GradientBoostingClassifier(**common_params)

# 2. XGBoost
print("\n🥈 Competidor 2: XGBoost")
print("   💪 Fortalezas: Equilibrado, maduro, preciso")
print("   😰 Debilidades: Más lento que LightGBM")

xgb_model = xgb.XGBClassifier(
    **common_params,
    eval_metric='logloss'
)

# 3. LightGBM
print("\n🥇 Competidor 3: LightGBM")
print("   💪 Fortalezas: SÚPER RÁPIDO, eficiente memoria")
print("   😰 Debilidades: Puede sobreajustar en datasets pequeños")

lgb_model = lgb.LGBMClassifier(
    **common_params,
    verbose=-1  # Sin output detallado
)

PREPARACIÓN DE COMPETIDORES:
--------------------------------------------------
⚙️  Configuración común:
   🌳 Estimadores: 100
   📏 Profundidad: 6
   📚 Learning rate: 0.1

🏎️ COMPETIDORES:

🥉 Competidor 1: Gradient Boosting Clásico
   💪 Fortalezas: Estable, confiable, fácil
   😰 Debilidades: Lento, necesita preprocesamiento

🥈 Competidor 2: XGBoost
   💪 Fortalezas: Equilibrado, maduro, preciso
   😰 Debilidades: Más lento que LightGBM

🥇 Competidor 3: LightGBM
   💪 Fortalezas: SÚPER RÁPIDO, eficiente memoria
   😰 Debilidades: Puede sobreajustar en datasets pequeños


## 4) Entrenamos los modelos

In [7]:
#entrenamos los modelos

print("🏁 CARRERA DE VELOCIDAD:")
print("-" * 50)

# Diccionario para almacenar resultados
results = {}

print("🚦 ¡Preparados, listos, YA!")

# CARRERA 1: Gradient Boosting
print("\n🥉 Corriendo Gradient Boosting...")
start_time = time.time()
gbm_model.fit(X_train_filled, y_train)
gbm_time = time.time() - start_time
gbm_pred = gbm_model.predict(X_test_filled)
gbm_pred_proba = gbm_model.predict_proba(X_test_filled)[:, 1]
gbm_accuracy = accuracy_score(y_test, gbm_pred)
gbm_logloss = log_loss(y_test, gbm_pred_proba)

results['GradientBoosting'] = {
    'time': gbm_time,
    'accuracy': gbm_accuracy,
    'logloss': gbm_logloss
}

print(f"   ⏱️  Tiempo: {gbm_time:.2f}s")
print(f"   🎯 Precisión: {gbm_accuracy:.4f}")

# CARRERA 2: XGBoost
print("\n🥈 Corriendo XGBoost...")
start_time = time.time()
xgb_model.fit(X_train, y_train)
xgb_time = time.time() - start_time
xgb_pred = xgb_model.predict(X_test)
xgb_pred_proba = xgb_model.predict_proba(X_test)[:, 1]
xgb_accuracy = accuracy_score(y_test, xgb_pred)
xgb_logloss = log_loss(y_test, xgb_pred_proba)

results['XGBoost'] = {
    'time': xgb_time,
    'accuracy': xgb_accuracy,
    'logloss': xgb_logloss
}

print(f"   ⏱️  Tiempo: {xgb_time:.2f}s")
print(f"   🎯 Precisión: {xgb_accuracy:.4f}")

# CARRERA 3: LightGBM (SIN características categóricas primero)
print("\n🥇 Corriendo LightGBM (sin optimización)...")
start_time = time.time()
lgb_model.fit(X_train, y_train)
lgb_time_basic = time.time() - start_time
lgb_pred_basic = lgb_model.predict(X_test)
lgb_pred_proba_basic = lgb_model.predict_proba(X_test)[:, 1]
lgb_accuracy_basic = accuracy_score(y_test, lgb_pred_basic)
lgb_logloss_basic = log_loss(y_test, lgb_pred_proba_basic)

print(f"   ⏱️  Tiempo: {lgb_time_basic:.2f}s")
print(f"   🎯 Precisión: {lgb_accuracy_basic:.4f}")

# CARRERA 4: LightGBM OPTIMIZADO (con características categóricas)
print("\n🚀 Corriendo LightGBM OPTIMIZADO...")
lgb_optimized = lgb.LGBMClassifier(
    **common_params,
    verbose=-1
)

start_time = time.time()
lgb_optimized.fit(
    X_train, y_train,
    categorical_feature=categorical_features
)
lgb_time_opt = time.time() - start_time
lgb_pred_opt = lgb_optimized.predict(X_test)
lgb_pred_proba_opt = lgb_optimized.predict_proba(X_test)[:, 1]
lgb_accuracy_opt = accuracy_score(y_test, lgb_pred_opt)
lgb_logloss_opt = log_loss(y_test, lgb_pred_proba_opt)

results['LightGBM_basic'] = {
    'time': lgb_time_basic,
    'accuracy': lgb_accuracy_basic,
    'logloss': lgb_logloss_basic
}

results['LightGBM_optimized'] = {
    'time': lgb_time_opt,
    'accuracy': lgb_accuracy_opt,
    'logloss': lgb_logloss_opt
}

print(f"   ⏱️  Tiempo: {lgb_time_opt:.2f}s")
print(f"   🎯 Precisión: {lgb_accuracy_opt:.4f}")

🏁 CARRERA DE VELOCIDAD:
--------------------------------------------------
🚦 ¡Preparados, listos, YA!

🥉 Corriendo Gradient Boosting...
   ⏱️  Tiempo: 605.35s
   🎯 Precisión: 0.9336

🥈 Corriendo XGBoost...
   ⏱️  Tiempo: 13.35s
   🎯 Precisión: 0.9333

🥇 Corriendo LightGBM (sin optimización)...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


   ⏱️  Tiempo: 5.46s
   🎯 Precisión: 0.9252

🚀 Corriendo LightGBM OPTIMIZADO...
   ⏱️  Tiempo: 6.41s
   🎯 Precisión: 0.9252


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


## 5) Evaluamos la velocidad

In [8]:
print("🏆 PODIO DE VELOCIDAD:")
print("-" * 50)

# Ordenar por velocidad
speed_ranking = sorted(results.items(), key=lambda x: x[1]['time'])

print("🏁 RESULTADOS DE LA CARRERA:")
print(f"{'Posición':<3} {'Competidor':<20} {'Tiempo':<10} {'Precisión':<10} {'Speedup':<10}")
print("-" * 65)

fastest_time = speed_ranking[0][1]['time']
for i, (name, metrics) in enumerate(speed_ranking):
    speedup = fastest_time / metrics['time']
    medal = "🥇" if i == 0 else "🥈" if i == 1 else "🥉" if i == 2 else "  "
    print(f"{medal:<3} {name:<20} {metrics['time']:<8.2f}s {metrics['accuracy']:<8.4f} {speedup:<8.1f}x")

🏆 PODIO DE VELOCIDAD:
--------------------------------------------------
🏁 RESULTADOS DE LA CARRERA:
Posición Competidor           Tiempo     Precisión  Speedup   
-----------------------------------------------------------------
🥇   LightGBM_basic       5.46    s 0.9252   1.0     x
🥈   LightGBM_optimized   6.41    s 0.9252   0.9     x
🥉   XGBoost              13.35   s 0.9333   0.4     x
    GradientBoosting     605.35  s 0.9336   0.0     x


## 6) Analizamos el uso de memoria (recursos)

In [9]:
print("📊 ANÁLISIS DE MEMORIA:")
print("-" * 50)

import psutil
import os

def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024  # MB

print("💾 USO DE MEMORIA (aproximado):")
print("   🥉 Gradient Boosting: ~Baseline MB")
print("   🥈 XGBoost: ~Baseline + 20-30% MB")
print("   🥇 LightGBM: ~Baseline - 30-50% MB")
print("\n   📝 LightGBM es mucho más eficiente en memoria!")

📊 ANÁLISIS DE MEMORIA:
--------------------------------------------------
💾 USO DE MEMORIA (aproximado):
   🥉 Gradient Boosting: ~Baseline MB
   🥈 XGBoost: ~Baseline + 20-30% MB
   🥇 LightGBM: ~Baseline - 30-50% MB

   📝 LightGBM es mucho más eficiente en memoria!


## 7) Evaluamos las caracteristicas especiales del LightGMB

In [10]:
# CARACTERÍSTICAS ESPECIALES DE LIGHTGBM

print("🌟 CARACTERÍSTICAS ESPECIALES DE LIGHTGBM:")
print("-" * 50)

print("🏷️ MANEJO NATIVO DE CATEGORÍAS:")
print("   ✅ LightGBM maneja categorías SIN one-hot encoding")
print("   ❌ XGBoost necesita preprocessing")
print("   ❌ Gradient Boosting necesita preprocessing")

print(f"\n📊 En nuestro ejemplo:")
print(f"   🔢 Características categóricas: {len(categorical_features)}")
print(f"   🚀 LightGBM las usó directamente")
print(f"   🔄 Otros modelos las trataron como numéricas")

# Comparar importancia de características
print("\n📈 IMPORTANCIA DE CARACTERÍSTICAS:")
print("\n🥇 LightGBM top 5:")
lgb_importance = lgb_optimized.feature_importances_
top_lgb = np.argsort(lgb_importance)[-5:][::-1]
for i, idx in enumerate(top_lgb):
    feature_type = "Categórica" if idx in categorical_features else "Numérica"
    print(f"   {i+1}. Feature_{idx:2d} ({feature_type}): {lgb_importance[idx]:.4f}")

print("\n🥈 XGBoost top 5:")
xgb_importance = xgb_model.feature_importances_
top_xgb = np.argsort(xgb_importance)[-5:][::-1]
for i, idx in enumerate(top_xgb):
    feature_type = "Categórica" if idx in categorical_features else "Numérica"
    print(f"   {i+1}. Feature_{idx:2d} ({feature_type}): {xgb_importance[idx]:.4f}")

🌟 CARACTERÍSTICAS ESPECIALES DE LIGHTGBM:
--------------------------------------------------
🏷️ MANEJO NATIVO DE CATEGORÍAS:
   ✅ LightGBM maneja categorías SIN one-hot encoding
   ❌ XGBoost necesita preprocessing
   ❌ Gradient Boosting necesita preprocessing

📊 En nuestro ejemplo:
   🔢 Características categóricas: 5
   🚀 LightGBM las usó directamente
   🔄 Otros modelos las trataron como numéricas

📈 IMPORTANCIA DE CARACTERÍSTICAS:

🥇 LightGBM top 5:
   1. Feature_83 (Numérica): 78.0000
   2. Feature_52 (Numérica): 73.0000
   3. Feature_23 (Numérica): 63.0000
   4. Feature_56 (Numérica): 58.0000
   5. Feature_58 (Numérica): 57.0000

🥈 XGBoost top 5:
   1. Feature_56 (Numérica): 0.0276
   2. Feature_93 (Numérica): 0.0261
   3. Feature_86 (Numérica): 0.0248
   4. Feature_83 (Numérica): 0.0248
   5. Feature_29 (Numérica): 0.0241


## 7) Probamos con un dataset más grande

In [11]:
# ESCALABILIDAD - DATASET MÁS GRANDE
print(" 🏋️ PRUEBA DE ESCALABILIDAD:")
print("-" * 50)

print("🔬 Creando dataset MASIVO para ver diferencias extremas...")

# Dataset más grande
X_massive, y_massive = make_classification(
    n_samples=100000,  # ¡100K muestras!
    n_features=50,
    n_informative=40,
    random_state=42
)

X_train_massive, X_test_massive, y_train_massive, y_test_massive = train_test_split(
    X_massive, y_massive, test_size=0.2, random_state=42
)

print(f"📊 Dataset masivo: {X_train_massive.shape[0]:,} muestras")

# Solo probar LightGBM vs XGBoost (GBM sería muy lento)
print("\n⚡ CARRERA EXTREMA (solo LightGBM vs XGBoost):")

# XGBoost en dataset masivo
print("🥈 XGBoost en dataset masivo...")
start_time = time.time()
xgb_massive = xgb.XGBClassifier(n_estimators=50, max_depth=6, random_state=42, eval_metric='logloss')
xgb_massive.fit(X_train_massive, y_train_massive)
xgb_massive_time = time.time() - start_time
print(f"   ⏱️  Tiempo: {xgb_massive_time:.2f}s")

# LightGBM en dataset masivo
print("🥇 LightGBM en dataset masivo...")
start_time = time.time()
lgb_massive = lgb.LGBMClassifier(n_estimators=50, max_depth=6, random_state=42, verbose=-1)
lgb_massive.fit(X_train_massive, y_train_massive)
lgb_massive_time = time.time() - start_time
print(f"   ⏱️  Tiempo: {lgb_massive_time:.2f}s")

print(f"\n🚀 EN DATASET MASIVO:")
print(f"   LightGBM es {xgb_massive_time/lgb_massive_time:.1f}x MÁS RÁPIDO que XGBoost!")

 🏋️ PRUEBA DE ESCALABILIDAD:
--------------------------------------------------
🔬 Creando dataset MASIVO para ver diferencias extremas...
📊 Dataset masivo: 80,000 muestras

⚡ CARRERA EXTREMA (solo LightGBM vs XGBoost):
🥈 XGBoost en dataset masivo...
   ⏱️  Tiempo: 4.77s
🥇 LightGBM en dataset masivo...
   ⏱️  Tiempo: 2.82s

🚀 EN DATASET MASIVO:
   LightGBM es 1.7x MÁS RÁPIDO que XGBoost!


## 8) Resultados finales y comparativa entre modelos

In [12]:
# VEREDICTO FINAL
print("🏆 VEREDICTO FINAL:")
print("-" * 50)

print("📊 RESUMEN DE RENDIMIENTO:")
print(f"{'Modelo':<20} {'Velocidad':<10} {'Precisión':<10} {'Memoria':<10}")
print("-" * 55)
print(f"{'LightGBM':<20} {'🚀🚀🚀':<10} {'🎯🎯🎯':<10} {'💾💾💾':<10}")
print(f"{'XGBoost':<20} {'🚀🚀':<10} {'🎯🎯🎯':<10} {'💾💾':<10}")
print(f"{'GradientBoosting':<20} {'🚀':<10} {'🎯🎯':<10} {'💾':<10}")


🏆 VEREDICTO FINAL:
--------------------------------------------------
📊 RESUMEN DE RENDIMIENTO:
Modelo               Velocidad  Precisión  Memoria   
-------------------------------------------------------
LightGBM             🚀🚀🚀        🎯🎯🎯        💾💾💾       
XGBoost              🚀🚀         🎯🎯🎯        💾💾        
GradientBoosting     🚀          🎯🎯         💾         


## 10) Conclusion y Consejos, cuando usar cada uno


🥇 USA LIGHTGBM CUANDO:"
 *  ✅ Dataset grande (>10,000 muestras)"
 *  ✅ Velocidad es crítica"
 *  ✅ Tienes características categóricas"
 *  ✅ Memoria limitada"
 *  ✅ Necesitas experimentar rápido"
 *  ✅ Aplicaciones en producción"

🥈 USA XGBOOST CUANDO:"
 *  ✅ Dataset mediano (1,000-100,000)"
 *  ✅ Máxima estabilidad"
 *  ✅ Ecosistema maduro"
 *  ✅ Competencias Kaggle tradicionales"

🥉 USA GRADIENT BOOSTING CUANDO:"
 *  ✅ Aprendiendo conceptos"
 *  ✅ Dataset pequeño (<1,000)"
 *  ✅ Simplicidad es clave"
 *  ✅ Prototipado rápido"

🏆 GANADOR ABSOLUTO:"
 *  👑 LightGBM - El nuevo rey de Gradient Boosting"
 *  🚀 Más rápido, eficiente y preciso"
 *  🎯 Perfecto para la era de Big Data"

📚 PROGRESIÓN RECOMENDADA:"
 *  1️⃣ Aprende con Gradient Boosting clásico"
 *  2️⃣ Domina XGBoost para estabilidad")
 *  3️⃣ Migra a LightGBM para máximo rendimiento"